# Optimize Prompts

This cookbook provides a look into an early version of OpenAI's prompt optimization system. Crafting effective prompts is a critical skill when working with AI models. Even expert users can inadvertently introduce contradictions, ambiguities, or inconsistencies that lead to suboptimal results. The system demonstrated here helps identify and fix common issues, resulting in more reliable, effective prompts.

The optimization process uses a multi-agent approach with specialized AI agents collaborating to analyze and rewrite prompts. The system automatically identifies and addresses several types of common issues:

- **Contradictions** in the prompt instructions
- **Missing or unclear format specifications**
- **Inconsistencies** between the prompt and few-shot examples

---

**Objective**: This notebook demonstrates how to automatically improve prompts by detecting and fixing common issues, helping you develop more robust and effective prompts for your applications.

**Cookbook Structure**  
This notebook follows this structure:

- [Step 1. System Overview](#1.-system-overview) - Learn how the prompt optimization system works  
- [Step 2. Data Models](#2.-data-models) - Understand the data structures used by the system
- [Step 3. Defining the Agents](#3.-defining-the-agents) - Meet the specialized agents that analyze and improve prompts
- [Step 4. Run Optimization Workflow](#4.run-optimization-workflow) - See how the system processes prompts
- [Step 5. Examples](#5.-examples) - Explore real-world examples of prompt optimization

**Prerequisites**
- The `openai` Python package 
- The `openai-agents` package
- An OpenAI API key set as `OPENAI_API_KEY` environment variable

## 1. System Overview

The prompt optimization system uses a collaborative multi-agent approach to analyze and improve prompts. Each agent specializes in detecting a specific type of issue:

1. **Dev-Contradiction-Checker**: Scans the prompt for logical contradictions or impossible instructions, like "only use positive numbers" and "include negative examples" in the same prompt.

2. **Format-Checker**: Identifies when a prompt expects structured output (like JSON, CSV, or Markdown) but fails to clearly specify the exact format requirements. This agent ensures that all necessary fields, data types, and formatting rules are explicitly defined.

3. **Few-Shot-Consistency-Checker**: Examines example conversations to ensure that the assistant's responses actually follow the rules specified in the prompt. This catches mismatches between what the prompt requires and what the examples demonstrate.

4. **Dev-Rewriter**: After issues are identified, this agent rewrites the prompt to resolve contradictions and clarify format specifications while preserving the original intent.

5. **Few-Shot-Rewriter**: Updates inconsistent example responses to align with the rules in the prompt, ensuring that all examples properly demonstrate the desired behavior.

By working together, these agents can systematically identify and fix issues that might otherwise lead to confusion or unexpected results when using the prompt with an AI model.

In [ ]:
# Import required modules
from openai import AsyncOpenAI
import asyncio
import json
import os
from enum import Enum
from typing import Any, List, Dict
from pydantic import BaseModel, Field
from agents import Agent, Runner, set_default_openai_client

openai_client: AsyncOpenAI | None = None

def _get_openai_client() -> AsyncOpenAI:
    global openai_client
    if openai_client is None:
        openai_client = AsyncOpenAI(
            api_key=os.environ.get("OPENAI_API_KEY", "your-api-key"),
        )
    return openai_client

set_default_openai_client(_get_openai_client())

## 2. Data Models

To facilitate structured communication between agents, the system uses Pydantic models to define the expected format for inputs and outputs. These models help validate data and ensure consistency throughout the optimization process.

The data models include:

1. **Role** - An enumeration for message roles (user/assistant)
2. **ChatMessage** - Represents a single message in a conversation
3. **Issues** - Base model for reporting detected issues
4. **FewShotIssues** - Extended model that adds rewrite suggestions for example messages
5. **MessagesOutput** - Contains optimized conversation messages
6. **DevRewriteOutput** - Contains the improved developer prompt

These models create a structured pipeline for analyzing prompts, detecting issues, and producing improved versions. Using Pydantic allows the system to validate that all data conforms to the expected format at each step of the process.

In [ ]:
class Role(str, Enum):
    """Role enum for chat messages."""
    user = "user"
    assistant = "assistant"

class ChatMessage(BaseModel):
    """Single chat message used in few-shot examples."""
    role: Role
    content: str

class Issues(BaseModel):
    """Structured output returned by checkers."""
    has_issues: bool
    issues: List[str]
    
    @classmethod
    def no_issues(cls) -> "Issues":
        return cls(has_issues=False, issues=[])

class FewShotIssues(Issues):
    """Output for few-shot contradiction detector including optional rewrite suggestions."""
    rewrite_suggestions: List[str] = Field(default_factory=list)
    
    @classmethod
    def no_issues(cls) -> "FewShotIssues":
        return cls(has_issues=False, issues=[], rewrite_suggestions=[])

class MessagesOutput(BaseModel):
    """Structured output returned by `rewrite_messages_agent`."""

    messages: list[ChatMessage]


class DevRewriteOutput(BaseModel):
    """Rewriter returns the cleaned-up developer prompt."""

    new_developer_message: str

## 3. Defining the Agents

In this section, we create the specialized AI agents that will analyze and improve prompts. Each agent is defined with a specific role, instructions, and output format. The `Agent` class is imported from the `openai-agents` package and provides a standardized way to create and run these specialized agents.

The agents are:

1. **dev_contradiction_checker** - Finds logical contradictions within the prompt
2. **format_checker** - Detects unclear or missing format specifications
3. **fewshot_consistency_checker** - Identifies mismatches between the prompt and examples
4. **dev_rewriter** - Rewrites the prompt to fix contradictions and format issues
5. **fewshot_rewriter** - Updates example responses to match prompt requirements

Each agent has carefully crafted instructions that guide its analysis. The instructions specify what to look for, what to ignore, and how to format the output. This level of specificity ensures that the agents work together effectively while focusing on their specific responsibilities.

In [ ]:
dev_contradiction_checker = Agent(
    name="contradiction_detector",
    model="gpt-4.1",
    output_type=Issues,
    instructions="""
    You are **Dev-Contradiction-Checker-v2**.

    Goal
    Detect *genuine* self-contradictions or impossibilities **inside** the developer prompt supplied in the variable `DEVELOPER_MESSAGE`.

    Definitions
    • A contradiction = two clauses that cannot both be followed.
    • Overlaps or redundancies in the DEVELOPER_MESSAGE are *not* contradictions.

    What you MUST do
    1. Compare every imperative / prohibition against all others.
    2. List at most FIVE contradictions (each as ONE bullet).
    3. If no contradiction exists, say so.

    Output format (**strict JSON**)
    Return **only** an object that matches the `Issues` schema:

    ```json
    {"has_issues": <bool>,
    "issues": [
        "<bullet 1>",
        "<bullet 2>"
    ]
    }
    - has_issues = true IFF the issues array is non-empty.
    - Do not add extra keys, comments or markdown.
""",
)
format_checker = Agent(
    name="format_checker",
    model="gpt-4.1",
    output_type=Issues,
    instructions="""
    You are Format-Checker-v2.

    Task
    Decide whether the developer prompt requires a structured output (JSON/CSV/XML/Markdown table, etc.).
    If so, flag any missing or unclear aspects of that format.

    Steps
    Categorise the task as:
    a. "conversation_only", or
    b. "structured_output_required".

    For case (b):
    - Point out absent fields, ambiguous data types, unspecified ordering, or missing error-handling.

    Do NOT invent issues if unsure. be a little bit more conservative in flagging format issues

    Output format
    Return strictly-valid JSON following the Issues schema:

    {
    "has_issues": <bool>,
    "issues": ["<desc 1>", "..."]
    }
    Maximum five issues. No extra keys or text.
""",
)
fewshot_consistency_checker = Agent(
    name="fewshot_consistency_checker",
    model="gpt-4.1",
    output_type=FewShotIssues,
    instructions="""
    You are FewShot-Consistency-Checker-v3.

    Goal
    Find conflicts between the DEVELOPER_MESSAGE rules and the accompanying **assistant** examples.

    USER_EXAMPLES:      <all user lines>          # context only
    ASSISTANT_EXAMPLES: <all assistant lines>     # to be evaluated

    Method
    Extract key constraints from DEVELOPER_MESSAGE:
    - Tone / style
    - Forbidden or mandated content
    - Output format requirements

    Compliance Rubric - read carefully
    Evaluate only what the developer message makes explicit.

    Objective constraints you must check when present:
    - Required output type syntax (e.g., "JSON object", "single sentence", "subject line").
    - Hard limits (length ≤ N chars, language required to be English, forbidden words, etc.).
    - Mandatory tokens or fields the developer explicitly names.

    Out-of-scope (DO NOT FLAG):
    - Whether the reply "sounds generic", "repeats the prompt", or "fully reflects the user's request" - unless the developer text explicitly demands those qualities.
    - Creative style, marketing quality, or depth of content unless stated.
    - Minor stylistic choices (capitalisation, punctuation) that do not violate an explicit rule.

    Pass/Fail rule
    - If an assistant reply satisfies all objective constraints, it is compliant, even if you personally find it bland or loosely related.
    - Only record an issue when a concrete, quoted rule is broken.

    Empty assistant list ⇒ immediately return has_issues=false.

    For each assistant example:
    - USER_EXAMPLES are for context only; never use them to judge compliance.
    - Judge each assistant reply solely against the explicit constraints you extracted from the developer message.
    - If a reply breaks a specific, quoted rule, add a line explaining which rule it breaks.
    - Optionally, suggest a rewrite in one short sentence (add to rewrite_suggestions).
    - If you are uncertain, do not flag an issue.
    - Be conservative—uncertain or ambiguous cases are not issues.

    be a little bit more conservative in flagging few shot contradiction issues
    Output format
    Return JSON matching FewShotIssues:

    {
    "has_issues": <bool>,
    "issues": ["<explanation 1>", "..."],
    "rewrite_suggestions": ["<suggestion 1>", "..."] // may be []
    }
    List max five items for both arrays.
    Provide empty arrays when none.
    No markdown, no extra keys.
    """,
)
dev_rewriter = Agent(
    name="dev_rewriter",
    model="gpt-4.1",
    output_type=DevRewriteOutput,
    instructions="""
    You are Dev-Rewriter-v2.

    You receive:
    - ORIGINAL_DEVELOPER_MESSAGE
    - CONTRADICTION_ISSUES (may be empty)
    - FORMAT_ISSUES (may be empty)

    Rewrite rules
    Preserve the original intent and capabilities.

    Resolve each contradiction:
    - Keep the clause that preserves the message intent; remove/merge the conflicting one.

    If FORMAT_ISSUES is non-empty:
    - Append a new section titled ## Output Format that clearly defines the schema or gives an explicit example.

    Do NOT change few-shot examples.

    Do NOT add new policies or scope.

    Output format (strict JSON)
    {
    "new_developer_message": "<full rewritten text>"
    }
    No other keys, no markdown.
""",
)
fewshot_rewriter = Agent(
    name="fewshot_rewriter",
    model="gpt-4.1",
    output_type=MessagesOutput,
    instructions="""
    You are FewShot-Rewriter-v2.

    Input payload
    - NEW_DEVELOPER_MESSAGE (already optimized)
    - ORIGINAL_MESSAGES (list of user/assistant dicts)
    - FEW_SHOT_ISSUES (non-empty)

    Task
    Regenerate only the assistant parts that were flagged.
    User messages must remain identical.
    Every regenerated assistant reply MUST comply with NEW_DEVELOPER_MESSAGE.

    After regenerating each assistant reply, verify:
    - It matches NEW_DEVELOPER_MESSAGE. ENSURE THAT THIS IS TRUE.

    Output format
    Return strict JSON that matches the MessagesOutput schema:

    {
    "messages": [
        {"role": "user", "content": "..."},
        {"role": "assistant", "content": "..."}
        ]
    }
    Guidelines
    - Preserve original ordering and total count.
    - If a message was unproblematic, copy it unchanged.
    """,
)

## 4.Run Optimization Workflow

Now that we've defined our agents and data models, let's examine how they work together to optimize prompts. The optimization workflow consists of several stages that run in parallel to efficiently analyze and improve prompts.

The following function implements the core workflow for prompt optimization. The process follows these steps:

1. **Parallel Analysis**: The function first runs the contradiction checker, format checker, and few-shot consistency checker in parallel (if examples are provided).

2. **Issue Collection**: Results from all checkers are collected and organized.

3. **Conditional Rewriting**: 
   - If contradiction or format issues are found, the prompt is rewritten using the dev_rewriter agent
   - If few-shot inconsistencies are found, the example responses are updated using the fewshot_rewriter agent

4. **Results Compilation**: The function returns a comprehensive result including the improved prompt, updated examples (if any), and details about the issues that were fixed.

This parallel approach makes the optimization process efficient, even when working with complex prompts and multiple examples.

In [ ]:
def _normalize_messages(messages: List[Any]) -> List[Dict[str, str]]:
    """Convert list of pydantic message models to JSON-serializable dicts."""
    result = []
    for m in messages:
        if hasattr(m, "model_dump"):
            result.append(m.model_dump())
        elif isinstance(m, dict) and "role" in m and "content" in m:
            result.append({"role": str(m["role"]), "content": str(m["content"])})
    return result

async def optimize_prompt_parallel(
    developer_message: str,
    messages: List["ChatMessage"],
) -> Dict[str, Any]:
    """
    Runs contradiction, format, and few-shot checkers in parallel,
    then rewrites the prompt/examples if needed.
    Returns a unified dict suitable for an API or endpoint.
    """

    # 1. Run all checkers in parallel (contradiction, format, fewshot if there are examples)
    tasks = [
        Runner.run(dev_contradiction_checker, developer_message),
        Runner.run(format_checker, developer_message),
    ]
    if messages:
        fs_input = {
            "DEVELOPER_MESSAGE": developer_message,
            "USER_EXAMPLES": [m.content for m in messages if m.role == "user"],
            "ASSISTANT_EXAMPLES": [m.content for m in messages if m.role == "assistant"],
        }
        tasks.append(Runner.run(fewshot_consistency_checker, json.dumps(fs_input)))

    results = await asyncio.gather(*tasks)

    # Unpack results
    cd_issues: Issues = results[0].final_output
    fi_issues: Issues = results[1].final_output
    fs_issues: FewShotIssues = results[2].final_output if messages else FewShotIssues.no_issues()

    # 3. Rewrites as needed
    final_prompt = developer_message
    if cd_issues.has_issues or fi_issues.has_issues:
        pr_input = {
            "ORIGINAL_DEVELOPER_MESSAGE": developer_message,
            "CONTRADICTION_ISSUES": cd_issues.model_dump(),
            "FORMAT_ISSUES": fi_issues.model_dump(),
        }
        pr_res = await Runner.run(dev_rewriter, json.dumps(pr_input))
        final_prompt = pr_res.final_output.new_developer_message

    final_messages: Union[List["ChatMessage"], List[Dict[str, str]]] = messages
    if fs_issues.has_issues:
        mr_input = {
            "NEW_DEVELOPER_MESSAGE": final_prompt,
            "ORIGINAL_MESSAGES": _normalize_messages(messages),
            "FEW_SHOT_ISSUES": fs_issues.model_dump(),
        }
        mr_res = await Runner.run(fewshot_rewriter, json.dumps(mr_input))
        final_messages = mr_res.final_output.messages

    return {
        "changes": True,
        "new_developer_message": final_prompt,
        "new_messages": _normalize_messages(final_messages),
        "contradiction_issues": "\n".join(cd_issues.issues),
        "few_shot_contradiction_issues": "\n".join(fs_issues.issues),
        "format_issues": "\n".join(fi_issues.issues),
        "general_improvements": "",
    }

## 5. Examples

Let's explore several practical examples to see how the prompt optimization system works in real-world scenarios. These examples demonstrate the system's ability to identify and fix different types of issues in prompts:

### Example 1: Fixing Contradictions

Our first example demonstrates how the system identifies and resolves contradictions in a prompt. This prompt contains instructions for parsing e-commerce product details from HTML, but it has contradictory rules for handling missing fields.

In [ ]:
async def example_contradiction():
    # A prompt with contradictory instructions
    prompt = """Quick-Start Card — Product Parser

Goal  
Digest raw HTML of an e-commerce product detail page and emit **concise, minified JSON** describing the item.

**Required fields:**  
name | brand | sku | price.value | price.currency | images[] | sizes[] | materials[] | care_instructions | features[]

**Extraction priority:**  
1. schema.org/JSON-LD blocks  
2. <meta> & microdata tags  
3. Visible DOM fallback (class hints: "product-name", "price")

** Rules:**  
- If *any* required field is missing, short-circuit with: `{"error": "FIELD_MISSING:<field>"}`.
- Prices: Numeric with dot decimal; strip non-digits (e.g., "1.299,00 EUR" → 1299.00 + "EUR").
- Deduplicate images differing only by query string. Keep ≤10 best-res.
- Sizes: Ensure unit tag ("EU", "US") and ascending sort.
- Materials: Title-case and collapse synonyms (e.g., "polyester 100%" → "Polyester").

**Sample skeleton (minified):**
```json
{"name":"","brand":"","sku":"","price":{"value":0,"currency":"USD"},"images":[""],"sizes":[],"materials":[],"care_instructions":"","features":[]}
Note: It is acceptable to output null for any missing field instead of an error ###"""
    
    result = await optimize_prompt_parallel(prompt, [])
    
    # Display the results
    if result["contradiction_issues"]:
        print("Contradiction issues:")
        print(result["contradiction_issues"])
        print()
        
    print("Optimized prompt:")
    print(result["new_developer_message"])
    
# Run the example
await example_contradiction()

### Example 2: Fixing Inconsistencies Between Prompt and Few-Shot Examples

Our second example shows how the system detects and fixes inconsistencies between a prompt and its accompanying examples. In this case, the prompt requires responses to be in JSON format, but one of the examples doesn't follow this requirement.

In [ ]:
async def example_fewshot_fix():
    prompt = "Respond **only** with JSON using keys `city` (string) and `population` (integer)."
    
    messages = [
        {"role": "user", "content": "Largest US city?"},
        {"role": "assistant", "content": "New York City"},
        {"role": "user", "content": "Largest UK city?"},
        {"role": "assistant", "content": "{\"city\":\"London\",\"population\":9541000}"}
    ]
    
    
    print("Few-shot examples before optimization:")
    print(f"User: {messages[0]['content']}")
    print(f"Assistant: {messages[1]['content']}")
    print(f"User: {messages[2]['content']}")
    print(f"Assistant: {messages[3]['content']}")
    print()
    
    # Call the optimization API
    result = await optimize_prompt_parallel(prompt, [ChatMessage(**m) for m in messages])
    
    # Display the results
    if result["few_shot_contradiction_issues"]:
        print("Inconsistency found:", result["few_shot_contradiction_issues"])
        print()
    
    # Show the optimized few-shot examples
    optimized_messages = result["new_messages"]
    print("Few-shot examples after optimization:")
    print(f"User: {optimized_messages[0]['content']}")
    print(f"Assistant: {optimized_messages[1]['content']}")
    print(f"User: {optimized_messages[2]['content']}")
    print(f"Assistant: {optimized_messages[3]['content']}")
    
# Run the example
await example_fewshot_fix()

### Example 3: Clarifying Formats in a Longer Prompt

Our final example shows how the system identifies and fixes unclear format specifications in a more complex prompt. This prompt asks for patent claim translation but doesn't clearly specify the required output format structure.

In [ ]:
async def example_format_issue():
    # A prompt with unclear or inconsistent formatting instructions
    prompt = """Task → Translate dense patent claims into 200-word lay summaries with a glossary.

Operating Steps:
1. Split the claim at semicolons, "wherein", or numbered sub-clauses.
2. For each chunk:
   a) Identify its purpose.
   b) Replace technical nouns with everyday analogies.
   c) Keep quantitative limits intact (e.g., "≥150 C").
3. Flag uncommon science terms with asterisks, and later define them.
4. Re-assemble into a flowing paragraph; do **not** broaden or narrow the claim's scope.
5. Omit boilerplate if its removal does not alter legal meaning.

Output should follow a Markdown template:
- A summary section.
- A glossary section with the marked terms and their definitions.

Corner Cases:
- If the claim is over 5 kB, respond with CLAIM_TOO_LARGE.
- If claim text is already plain English, skip glossary and state no complex terms detected.

Remember: You are *not* providing legal advice—this is for internal comprehension only."""

    # Call the optimization API to check for format issues
    result = await optimize_prompt_parallel(prompt, [])

    # Display the results
    if result.get("format_issues"):
        print("Format issues found:", result["format_issues"])
        print()

    print("Optimized prompt:")
    print(result["new_developer_message"])

# Run the example
await example_format_issue()

## 6. Conclusion and Best Practices

This notebook has demonstrated an early version of OpenAI's prompt optimization system, which uses specialized AI agents to automatically detect and fix common issues in prompts. The system focuses on three key areas:

1. **Detecting contradictions**: Finding and resolving instructions that conflict with each other
2. **Clarifying format specifications**: Ensuring that output format requirements are clear and comprehensive
3. **Aligning examples with requirements**: Making sure that example responses match the rules in the prompt

By applying these optimizations, you can create more effective prompts that lead to more consistent, reliable results from AI models.

### Best Practices for Prompt Engineering

Based on the patterns and fixes we've seen in this notebook, here are some best practices to follow when creating prompts:

1. **Check for contradictions**: Review your prompt carefully to ensure that your instructions don't contradict each other.

2. **Be explicit about formats**: If you expect a specific output format, clearly define the structure, field requirements, and how to handle edge cases.

3. **Provide consistent examples**: Ensure that your few-shot examples follow all the rules specified in your prompt.

4. **Define key terms**: Clearly define any important terms, labels, or categories that might be ambiguous.

5. **Specify error handling**: Provide clear instructions for how the model should handle edge cases or errors.

6. **Organize with clear structure**: Use headings, bullet points, and numbering to make your prompt easy to follow.

By following these practices and using tools like the optimization system demonstrated in this notebook, you can create more effective prompts that lead to better results from AI models.